Greedy Solve
============

In [1]:
def compute_sorted_elements(stack):
    if len(stack)==0: return 0
    sorted_elements=1
    while(sorted_elements<len(stack) and stack[sorted_elements] <= stack[sorted_elements-1]):
        sorted_elements +=1
    
    return sorted_elements

def gvalue(stack):
    if len(stack)==0: return 100
    else: return stack[-1]
    

In [2]:
class Layout:
    def __init__(self, stacks, H):
        self.stacks = stacks
        self.sorted_elements = []
        self.sorted_stack = []
        self.unsorted_stacks = 0
        self.steps = 0
        self.current_step = 0
        self.moves = []
        self.H = H
        j=0
        
        for stack in stacks:
            self.sorted_elements.append(compute_sorted_elements(stack))
            if not self.is_sorted_stack(j):
                self.unsorted_stacks += 1
                self.sorted_stack.append(False)
            else: self.sorted_stack.append(True)
            j += 1
    
    def move(self,i,j):
        c = self.stacks[i][-1]
        if self.is_sorted_stack(i):
            self.sorted_elements[i] -= 1
            
        if self.is_sorted_stack(j) and gvalue(self.stacks[j]) >= c:
            self.sorted_elements[j] += 1
            
        self.stacks[i].pop()
        self.stacks[j].append(c)
        self.is_sorted_stack(i)
        self.is_sorted_stack(j)
        self.steps += 1
        self.current_step += 1
        self.moves.append((i,j))
        
    def highlighted_print(self, stack_set):
        for stack in self.stacks:
            if stack in stack_set:
                print('\033[94m'+str(stack)+'\033[0m')
            else:
                print(stack)
    
    def next(self, printed=False):
        if self.current_step == self.steps: 
            if printed: self.highlighted_print([])
            return
        i,j = self.moves[self.current_step]
        self.current_step += 1
        c=self.stacks[i].pop()
        self.stacks[j].append(c)
        
        if printed:
            s=[]
            if(self.current_step < self.steps):
                i,j = self.moves[self.current_step]
                s.append(self.stacks[i]), s.append(self.stacks[j])
            self.highlighted_print(s)
                

    def prev(self, printed=False):
        if self.current_step == 0:
            if printed: self.highlighted_print([])
            return
        
        self.current_step -= 1
        j,i = self.moves[self.current_step]
        c=self.stacks[i].pop()
        self.stacks[j].append(c)

        if printed:   
            s=[]
            if(self.current_step-1 > 0):
                i,j = self.moves[self.current_step-1]
                s.append(self.stacks[i]), s.append(self.stacks[j])
            self.highlighted_print(s)
        
    def is_sorted_stack(self, j):
        sorted = len(self.stacks[j]) == self.sorted_elements[j]
        if j<len(self.sorted_stack) and self.sorted_stack[j] != sorted: 
            self.sorted_stack[j] = sorted
            if sorted == True: self.unsorted_stacks -= 1
            else: self.unsorted_stacks += 1
        return sorted
    
    
        

In [3]:
def read_file(file, H):
    with open(file) as f:
        S, C = [int(x) for x in next(f).split()] # read first line
        stacks = []
        for line in f: # read rest of lines
            stack = [int(x) for x in line.split()[1::]]
            #if stack[0] == 0: stack.pop()
            stacks.append(stack)
            
        layout = Layout(stacks,H)
    return layout

In [4]:
def select_destination_stack(layout, orig):
    s_o = layout.stacks[orig]
    c = s_o[-1]
    best_eval=-1000000;
    best_dest=None
    dest=-1;

    for dest in range(len(layout.stacks)):
            if orig==dest: continue
            s_d = layout.stacks[dest]

            if(layout.H == len(s_d)): continue
            top_d=gvalue(s_d)

            ev=0

            if layout.is_sorted_stack(dest) and c<=top_d:
              #c can be well-placed: the sorted stack minimizing top_d is preferred.
              ev = 10000 - 100*top_d
            elif not layout.is_sorted_stack(dest) and c>=top_d:
              #unsorted stack with c>=top_d maximizing top_d is preferred
              ev = top_d
            elif layout.is_sorted_stack(dest):
              #sorted with minimal top_d
              ev = -100 - top_d
            else:
              #unsorted with minimal numer of auxiliary stacks
              ev = -10000 #+ required_stacks(dest)

            if ev > best_eval:
                best_eval=ev
                best_dest=dest

    return best_dest

In [5]:
def reachable_height(layout, i):
    if not layout.is_sorted_stack(i): return -1;
    
    h = len(layout.stacks[i])
    if h==layout.H: return size;
    
    stack=layout.stacks[i]
    all_stacks = True #True: all the bad located tops can be placed in stack
    
    for k in range(len(layout.stacks)):
        if k==i: continue
        if layout.is_sorted_stack(k): continue
            
        stack_k=layout.stacks[k]
        unsorted = len(stack_k)-layout.sorted_elements[k]
        prev = 1000;
        for j in range (1,unsorted+1):
            if stack_k[-j] <= prev and stack_k[-j] <=j:
                h += 1
                if h==layout.H: return h
                prev = stack_k[-j]
            else: 
                if j==1: all_stacks=False
                break
                
    if all_stacks: return layout.H
    else: return h
             

In [6]:
def SF_move(layout):
    s_o = None
    s_d = None
    min_dif = 10000
    for i in range(len(layout.stacks)):
        if(layout.is_sorted_stack(i) and len(layout.stacks[i]) < layout.H):
            top = gvalue(layout.stacks[i])
            for k in range(len(layout.stacks)):
                if k!=i and not layout.is_sorted_stack(k):
                    if layout.stacks[k][-1] <= top and (top - layout.stacks[k][-1]) < min_dif:
                        min_dif = top - layout.stacks[k][-1]
                        s_d=i; s_o = k
    if s_o != None: 
        layout.move(s_o,s_d)
        return True
    return False

In [7]:
def SD_move(layout):
    best_ev = 0
    for i in range(len(layout.stacks)):
        prom = sum(layout.stacks[i]) / len(layout.stacks[i]) 
        ev = 10000 - 100*len(layout.stacks[i]) - prom
        if ev > best_ev:
            best_ev = ev
            s_o = i
            
    while len(layout.stacks[s_o])>0:
        s_d = select_destination_stack(layout,s_o)
        layout.move(s_o,s_d)
        if reachable_height(layout,s_o)==layout.H: return    

In [8]:
def greedy_solve(layout):
    while layout.unsorted_stacks>0:
        if not SF_move(layout):
            SD_move(layout)
    return layout.steps

In [9]:
def solve_file(file,H):
    layout = read_file(file,H)
    greedy_solve(layout)
    return layout

Usando el Greedy_Solve
--------------------

Para resolver un problema basta con crear el layout como una lista de pilas: 

In [10]:
stacks=[[12],
 [3, 10, 8, 15, 14, 11, 6],
 [8, 13],
 [9, 15, 4, 6, 4],
 [4, 12, 10, 6],
 [14, 16, 9, 1, 12, 15, 3, 5],
 [13, 6],
 [5, 11, 12, 2, 7],
 [8, 10, 3, 8, 7, 1, 15],
 [9, 10, 11, 14, 6, 4, 11, 2],
 [16, 4],
 [7, 9, 4, 12, 2],
 [2, 15, 5, 6, 13, 12, 1, 5],
 [7, 13, 3, 14],
 [12, 1, 14, 16, 2, 5, 16, 13],
 [7]]

Luego creamos el layout pasando las *pilas* y la **altura máxima H**

In [11]:
layout = Layout(stacks, 8) #el numero es la altura máxima

Para resolver llamamos a la función greedy_solve la que nos retorna la cantidad de pasos que realizó

In [12]:
greedy_solve(layout)

64

In [13]:
#Podemos ver el layout final
layout.stacks

[[12, 11, 10, 7, 7, 7, 3, 2],
 [16, 15],
 [8, 8, 8, 7, 6, 5, 4, 3],
 [16, 15, 13, 12, 12, 12, 12, 11],
 [4, 3, 1],
 [14, 14, 11, 10],
 [13, 6, 6, 6, 6, 5, 5, 1],
 [5, 5, 4, 1],
 [8, 6],
 [9, 9],
 [16, 4, 4, 4, 3, 2, 2, 2],
 [7],
 [2],
 [16, 14, 13, 11, 10, 10, 9, 9],
 [12, 1],
 [15, 15, 15, 14, 14, 13, 13, 12]]

In [14]:
# Y también retroceder hasta el comienzo para poder ver los paso uno a uno
while layout.current_step-1 >0: 
    layout.prev()

In [15]:
#Luego de quedar al comienzo, cada vez que ejecutamos esto avanzamos un paso
layout.next(True)

[12]
[3, 10, 8, 15, 14, 11]
[8, 13]
[9, 15, 4, 6, 4]
[4, 12, 10]
[14, 16, 9, 1, 12, 15, 3, 5]
[13, 6, 6, 6]
[5, 11, 12, 2, 7]
[8, 10, 3, 8, 7, 1, 15]
[9, 10, 11, 14, 6, 4, 11, 2]
[16, 4]
[7, 9, 4, 12, 2]
[2, 15, 5, 6, 13, 12, 1, 5]
[7, 13, 3, 14]
[12, 1, 14, 16, 2, 5, 16, 13]
[7]


Tk

In [16]:
#Retornamos al primer layout
while layout.current_step-1 >0: 
    layout.prev()
    
layout.prev();

In [17]:

from tkinter import * 
import tkinter

def op():
    labelframe=LabelFrame(top, text="")
    labelframe1=labelframe
    return labelframe

#/////////////////////////////////////////    
top = Tk()  
#top.geometry("800x500")  
#Se obtiene tamaño de la pantalla
windowWidth = top.winfo_reqwidth()
windowHeight = top.winfo_reqheight()
positionRight = int(top.winfo_screenwidth()/4 - windowWidth/4)
positionDown = int(top.winfo_screenheight()/4 - windowHeight/4)

top.geometry("+{}+{}".format(positionRight, positionDown))
#Se crean los panels

#columnas ahora con colores
labelframe1 = LabelFrame(top, text="")
labelframe1.grid(padx=10, pady=10,columnspan=len(layout.stacks), rowspan=layout.H)

#h=0
#for i in range(len(layout.stacks)):
#    s=layout.stacks[i]
#    #print(s)
#    blank=layout.H-len(s)
#    for l in range(len(s),blank):
#        tkinter.Label(labelframe1, text="-",borderwidth=10 ).grid(row=ff,column=i)
#        print(len(s))
#        print(blank)
#        ff=ff+1
#    for j in range(len(s)):
#        sobras=h%2
#        if sobras == 0:
#            tkinter.Label(labelframe1, text=str(h),borderwidth=10 ).grid(row=j,column=i)
#        else:
#            tkinter.Label(labelframe1, text=str(h),borderwidth=10,fg="green" ).grid(row=j,column=i)
#            
#        #tkinter.Label(labelframe1, text=str(i)+","+str(j),borderwidth=10 ).grid(row=j,column=i)
#        h=h+1

for i in range(len(layout.stacks)):
    #se obtiene el stack
    s=layout.stacks[i]
    #color
    aux=24500
    cont=1
    for item in s:
        if item <= aux:
            aux=item
            cont+=1
        else:
            break
    cont+=-1
    #Color de la columna
    color=len(s)-cont
    #/////////
    height=layout.H-len(s)
    #print(color)
    j=0
    #Aqui se imprimen los cuadros vacios para que no quede al reves el visual
    while j < height:
        tkinter.Label(labelframe1, text="",borderwidth=10, bg="white", padx=9, pady=1  ).grid(row=j,column=i)
        j+=1
    
    #Se imprimen en pantalla los valores
    for item in reversed(s):
        if color>0:
            if item > 9:
                tkinter.Label(labelframe1, text=str(item),borderwidth=1,relief="solid",padx=10,pady=10,bg="red").grid(row=j,column=i)
            else:
                tkinter.Label(labelframe1, text=str(item),borderwidth=1,relief="solid",padx=13, pady=10,bg="red").grid(row=j,column=i)
            color+=-1
        else:
            if item>9:
                tkinter.Label(labelframe1, text=str(item),borderwidth=1,relief="solid",padx=10, pady=10,bg="cyan" ).grid(row=j,column=i)
            else:
                tkinter.Label(labelframe1, text=str(item),borderwidth=1,relief="solid",padx=13, pady=10,bg="cyan" ).grid(row=j,column=i)
        #veamos si se puede pintar
        j+=1


#layout.H tiene la altura
#layout.moves tiene los movimientos
#layout.stacks tiene los stacks


#BOTONES
labelframe2 = LabelFrame(top, text="")
labelframe2.grid(padx=20, pady=20,columnspan=len(layout.stacks), rowspan=1)

#button = tkinter.Button(labelframe2, text="prev",padx=13, command=op)
#button.grid(row=0, column=1)

#button2 = tkinter.Button(labelframe2, text="next",padx=13)
#button2.grid(row=0, column=6)
#/////////////////////////////////////////////////////////////////
top.mainloop()  

In [ ]:
import gurobipy as gp
from gurobipy import GRB
import copy 

#Pasos previos
#Se copia la instancia
instancia=copy.deepcopy(stacks)


bp=[]
#Se rellenan los vacios.
for i in range(len(stacks)):
    s=instancia[i]
    h=layout.H-len(s)
    #Se buscan los items mal colocados unicamente
    cont=1
    aux=24500
    for item in s:
        if item <= aux:
            aux=item
            cont+=1
        else:
            break
    cont+=-1
    bp.append(cont)
    #El resto se rellena con ceros
    #j=0;
    #while j<h:
    #    s.append(0)
    #    j=j+1;


#Nueva version simplificada del layout
simpl=[]

for i in range(len(stacks)):
    s=instancia[i]
    aux=bp[i]
    nuevo=[]
    for j in range(aux-1,len(s)):
        nuevo.append(s[j])
    simpl.append(nuevo)
        
    
    
#Se continua con gurobi

n = layout.H
c = len(instancia)


bottom=[]

for i in range(len(simpl)):
    s=simpl[i]
    bottom.append(s[0])
    s.pop(0)
print("Entrada")    
for i in instancia:
    print (i)
print("\n//****************************************************\n")
print("Problema simplificado")
for i in simpl:
    print (i)
print("\n\n")
print("base")
print(bottom)


#Restricciones

#hacia donde se puede mover
id=1
print("\n//****************************************************\n")
for i in range(len(simpl)):
    s=simpl[i]
    if len(s) != 0:
        #print(s)
        for j in range(len(s)):
            #print (s[j])
            string=""
            for k in range(len(bottom)):
                if s[j]<=bottom[k] and k != i:
                    string+="X"+str(id)+str(k+1)+"+"
            string = string[:len(string) - 1]
            if len(string)>1:
                string+="=1"
                print(string)
            else:
                print("No se puede realizar como se desea, so F")
            id+=1
print("\n//****************************************************\n")
for i in range(len(stacks)):
    s=stacks[i]
    alt=h+1-len(s)
    print ("sum(xi"+str(i+1)+")<="+str(alt))
    
print("\n//****************************************************\n")


#DECLARAMOS EL MODELO
model = gp.Model("CPMP")

#Se crea una matriz de variables nxc
vars = model.addVars(n, c,vtype=GRB.BINARY, name='layout')

#Se updatea el modelo, porsiacaso
model.update

#No hay una funcion objetivo por lo que no se coloca

#Constraints]
#model.addConstr(x <= 1, name='c0')

print (vars[0,0])

for i in range(n):
    for j in range(c):
        model.addConstr(vars[i,j])
        
